In [1]:
import ee # Google Earth Engine
import datetime
#import ipyleaflet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
# from shapely.geometry import MultiPolygon, Polygon
from shapely.geometry import box
import warnings
from functools import reduce

Initialize Google Earth Engine (GEE)

In [2]:
ee.Initialize()

Specify an area of interest

In [3]:
# area of interest has lat, lon of (-10.883689, -44.005800)
Brazil_ex1_lat, Brazil_ex1_lon = -10.883689, -44.005800
Brazil_ex1_edge_len = 0.2

In [4]:
US_IL_lat, US_IL_lon = 40.707570, -88.804750
US_IL_edge_len = 0.2

In [5]:
US_ID_lat, US_ID_lon = 43.771114, -116.736866
US_ID_edge_len = 0.005

In [38]:
India_1_lat, India_1_lon = 23.967052, 72.400000
India_1_edge_len = 0.005

Define the function to plot satellite images or pull satellite image data

In [24]:
def satellite_imagery(source, center_lat, center_lon, edge_len,
                      start_date, end_date, 
                      plot_option, time_series):
    # Sentinel-2 Level 1-C: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2
    # Sentinel-2 Level 2-A: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    # Landsat Tier structure: https://developers.google.com/earth-engine/landsat#landsat-collection-structure
    if source == "Sentinel2_1C":
        source_loc = 'COPERNICUS/S2'
    elif source == "Sentinel2_2A":
        source_loc = 'COPERNICUS/S2_SR'
    elif source == "Landsat7":
        source_loc = 'LANDSAT/LE07/C01/T1'
    elif source == "Landsat8":
        source_loc = 'LANDSAT/LC08/C01/T1_SR'
#     elif source == "Modis":
#         source_loc = 'MODIS/006/MOD13A2'
    else:
        raise Exception('Invalid source of satellite imagery')
    
    # Specify area of interest (one for ee.Geometry, one for shapely)
#     area_of_interest_ee = ee.Geometry.Polygon([[center_lon-edge_len/2, center_lat-edge_len/2], 
#                            [center_lon-edge_len/2, center_lat+edge_len/2], 
#                            [center_lon+edge_len/2, center_lat+edge_len/2], 
#                            [center_lon+edge_len/2, center_lat-edge_len/2]])
    area_of_interest_ee = ee.Geometry.Rectangle([center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2])
    
#     area_of_interest_shapely = Polygon([[center_lon-edge_len/2, center_lat-edge_len/2], 
#                            [center_lon-edge_len/2, center_lat+edge_len/2], 
#                            [center_lon+edge_len/2, center_lat+edge_len/2], 
#                            [center_lon+edge_len/2, center_lat-edge_len/2]])
    area_of_interest_shapely = box(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
    
    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection(source_loc)
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest_ee))
    
    # Remove tiles with high cloud coverage
    if 'Sentinel' in source:
        # Remove image that's too small (likely to be partial image)
        # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
        img_collect = img_collect.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",20) 
#                                 .filter(ee.Filter.gt('system:asset_size', 1200000000))
        img_collect_no_partial = img_collect.filter(ee.Filter.gt('system:asset_size', 1200000000))
    elif 'Landsat' in source:
        img_collect = img_collect.filter(ee.Filter.lt('CLOUD_COVER', 3))
    
    assert (img_collect.size().getInfo()>0), "No valid image"
    print("Total number of images in the collection: ", img_collect.size().getInfo())
        
    if 'Sentinel' in source:
        # Extract tile information from each image
        # Note: tiles can overlap a little bit
        unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
        if len(unique_tiles) > 1:
            warnings.warn('Multiple tiles are selected. Proceed with caution.')
            print('Number of tiles selected: ', len(unique_tiles))
        if img_collect_no_partial.size().getInfo() < img_collect.size().getInfo():
            warnings.warn('There are partial images in the collection. Proceed with caution.')
            print('Number of partial images: ', img_collect.size().getInfo()-img_collect_no_partial.size().getInfo())
        
    if 'Sentinel' in source:
        # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
        band_blue = 'B2' #10m
        band_green = 'B3' #10m
        band_red = "B4"  #10m
        band_nir = 'B8'  #10m
    # Reference: https://landsat.gsfc.nasa.gov/landsat-data-continuity-mission/
    elif 'Landsat7' in source:
        # Reference: https://www.usgs.gov/land-resources/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con
        band_blue = 'B1' #30m
        band_green = 'B2' #30m
        band_red = "B3"  #30m
        band_nir = 'B4'  #30m
    elif 'Landsat8' in source:
        # Reference: https://www.usgs.gov/faqs/what-are-best-landsat-spectral-bands-use-my-research?qt-news_science_products=0#qt-news_science_products
        band_blue = 'B2' #30m
        band_green = 'B3' #30m
        band_red = "B4"  #30m
        band_nir = 'B5'  #30m
    
    def calc_NDVI(img):
        ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(ndvi)
        return composite
    
    # SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
    def calc_SAVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        savi =  ee.Image(img.expression(
            '(1 + L) * float(nir - red)/ (nir + red + L)',
            {
                'nir': img.select(band_nir),
                'red': img.select(band_red),
                'L': 0.5
            })).rename(["savi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(savi)
        return composite

    # EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
    #     C1=6, C2=7.5, and L=1
    def calc_EVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        evi = ee.Image(img.expression(
          '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
          {   
              'nir': img.select(band_nir),
              'red': img.select(band_red),
              'blue': img.select(band_blue),
              'L': 0.2,
              'C1': 6,
              'C2': 7.5
          })).rename(["evi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(evi)
        return composite
    
    def calc_YYYYMM(img):
        return img.set('YYYYMM', img.date().format("YYYYMM"))
    
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.median()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                        data = ee_object.getInfo(),
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                        tiles = map_id_dict['tile_fetcher'].url_format,
                        attr = 'Google Earth Engine',
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    
    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI)
    
    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    
    if len(unique_month) > 0:
        warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
    
    img_calc_month_dict = dict()
    temp_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median()
        img_calc_month2 = img_calc_month_dict[month].addBands(ee.Image.pixelLonLat())
        data_month_lst = img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
                                                             geometry=area_of_interest_ee, maxPixels=1e13, scale=10)
        lat_series = pd.Series(np.array((ee.Array(data_month_lst.get("latitude")).getInfo())), name="lat")
        lon_series = pd.Series(np.array((ee.Array(data_month_lst.get("longitude")).getInfo())), name="lon")
        ndvi_series = pd.Series(np.array((ee.Array(data_month_lst.get("ndvi")).getInfo())), name=month+'_NDVI')
        savi_series = pd.Series(np.array((ee.Array(data_month_lst.get("savi")).getInfo())), name=month+'_SAVI')
        evi_series = pd.Series(np.array((ee.Array(data_month_lst.get("evi")).getInfo())), name=month+'_EVI')
        temp_dict[month] = pd.concat([lat_series, lon_series, ndvi_series, savi_series, evi_series], axis=1)
    
    df_lst = list(temp_dict.values())
    out_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), df_lst)
    if plot_option == 'no':
        # Output the column names that have null values
        if len(out_df.columns[out_df.isnull().any()]) > 0:
            warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
        return out_df
    else:
        # Create a folium map object.
        myMap = folium.Map(location=[center_lat, center_lon], zoom_start=8)
        # Add the box around the area of interest
        folium.GeoJson(area_of_interest_shapely, name="Area of Interest").add_to(myMap)

        if plot_option == 'RGB':
            if 'Sentinel' in source:
                visParams = {'min':0, 'max':3000}
            elif 'Landsat' in source:
                visParams = {'min':0, 'max':255}
            if time_series=='no':
                myMap.add_ee_layer(img_collect_calc.median().select(band_red, band_green, band_blue), visParams, name=source+' '+plot_option)
            elif time_series=="monthly":
                for month in unique_month:
                    myMap.add_ee_layer(img_calc_month_dict[month].select(band_red, band_green, band_blue), visParams, name=source+' '+plot_option+' '+month)
        elif plot_option == 'NDVI':
            visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
            if time_series=='no':
                myMap.add_ee_layer(img_collect_calc.select("ndvi"), visParams, name=source+' '+plot_option)
            elif time_series=="monthly":
                for month in unique_month:
                    myMap.add_ee_layer(img_calc_month_dict[month].select("ndvi"), visParams, name=source+' '+plot_option+' '+month)
        elif plot_option == 'SAVI':
            visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
            if time_series=='no':
                myMap.add_ee_layer(img_collect_calc.median().select("savi"), visParams, name=source+' '+plot_option)
            elif time_series=="monthly":
                for month in unique_month:
                    myMap.add_ee_layer(img_calc_month_dict[month].select("savi"), visParams, name=source+' '+plot_option+' '+month)
        elif plot_option == 'EVI':
            visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
            if time_series=='no':
                myMap.add_ee_layer(img_collect_calc.median().select("evi"), visParams, name=source+' '+plot_option)
            elif time_series=="monthly":
                for month in unique_month:
                    myMap.add_ee_layer(img_calc_month_dict[month].select("evi"), visParams, name=source+' '+plot_option+' '+month)

        # Add a layer control panel to the map.
        myMap.add_child(folium.LayerControl())

        return myMap

Define a function to convert Pandas DataFrame outputted from `satellite_imagery(..., plot_option='no')` to numpy array

In [7]:
def time_series_prep(in_df, stat_option):
    col_lst = list(in_df.columns)
    col_lst_select = [i for i in col_lst if stat_option in i]
    # Rearrange the order the columns based on YYYYMM
    col_lst_select.sort()
    out_df = in_df[col_lst_select]
    return out_df.to_numpy()

Display Landsat 7 RGB satellite image for a Brazil cropland

In [111]:
# satellite_imagery(source="Landsat7", center_lat=Brazil_ex1_lat, center_lon=Brazil_ex1_lon, edge_len=Brazil_ex1_edge_len,
#                   start_date='2018-05-01', end_date='2018-7-31',
#                   plot_option='RGB', time_series="no")

Display Sentinel-2 RGB satellite image for a Brazil cropland (higher resolution compared to Landsat 7 image with not fainted stripes)

In [113]:
# satellite_imagery(source="Sentinel2_1C", center_lat=Brazil_ex1_lat, center_lon=Brazil_ex1_lon, edge_len=Brazil_ex1_edge_len,
#                   start_date='2018-05-01', end_date='2018-7-31',
#                   plot_option='RGB', time_series="no")

Display Sentinel-2 NDVI satellite image for a Brazil cropland (red means no/unhealthy vegetation and green means healthy vegetation)

In [130]:
# satellite_imagery(source="Sentinel2_1C", center_lat=Brazil_ex1_lat, center_lon=Brazil_ex1_lon, edge_len=Brazil_ex1_edge_len,
#                   start_date='2018-05-01', end_date='2018-5-10',
#                   plot_option='NDVI', time_series="no")

Display Sentinel-2 NDVI satellite image for IL croplands.  
Filtering (e.g., cloud cover filter) is done at a tile level, so different tiles can have different number of images. This is the cause of red trapezoid on the map.

In [182]:
# satellite_imagery(source="Sentinel2_1C", center_lat=US_IL_lat, center_lon=US_IL_lon, edge_len=US_IL_edge_len,
#                   start_date='2018-05-01', end_date='2018-7-31',
#                   plot_option='NDVI', time_series="no")

Display Sentinel-2 NDVI satellite image for IL croplands by month.

In [143]:
# satellite_imagery(source="Sentinel2_1C", center_lat=US_IL_lat, center_lon=US_IL_lon, edge_len=US_IL_edge_len,
#                   start_date='2018-1-01', end_date='2018-12-31',
#                   plot_option='NDVI', time_series="monthly")

Display an NDVI example of pivot irrigation system (in Idaho)

In [27]:
# satellite_imagery(source="Sentinel2_1C", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
#                   start_date='2018-1-01', end_date='2018-12-31',
#                   plot_option='NDVI', time_series="no")

Display an EVI example of pivot irrigation system (in Idaho)

In [45]:
# satellite_imagery(source="Sentinel2_1C", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
#                   start_date='2018-1-01', end_date='2018-12-31',
#                   plot_option='EVI', time_series="no")

Display an SAVI example of pivot irrigation system (in Idaho)

In [49]:
# satellite_imagery(source="Sentinel2_1C", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
#                   start_date='2018-1-01', end_date='2018-12-31',
#                   plot_option='SAVI', time_series="no")

Display an NDVI example of pivot irrigation system by month (in Idaho)  
Preliminary finding: irrigated croplands have lower NDVI in the winter and higher NDVI in the summer.

In [40]:
# satellite_imagery(source="Sentinel2_1C", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
#                   start_date='2018-1-01', end_date='2018-12-31',
#                   plot_option='NDVI', time_series="monthly")

================================================================================================================

In [25]:
satellite_imagery(source="Sentinel2_1C", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
                  start_date='2018-1-01', end_date='2018-12-31',
                  plot_option='NDVI', time_series="monthly")

Total number of images in the collection:  27


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:171: UserWarning: There are null values in the output DataFrame. Proceed with caution.


In [19]:
# satellite_imagery(source="Modis", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
#                   start_date='2018-1-01', end_date='2018-12-31',
#                   plot_option='NDVI', time_series="no")

In [20]:
ID_df = satellite_imagery(source="Sentinel2_1C", center_lat=US_ID_lat, center_lon=US_ID_lon, edge_len=US_ID_edge_len,
                  start_date='2018-1-01', end_date='2018-12-31',
                  plot_option='no', time_series="monthly")

Total number of images in the collection:  27


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:167: UserWarning: There are null values in the output DataFrame. Proceed with caution.


In [21]:
ID_df.to_csv('time_series_ID_scale10.csv')

In [22]:
ID_df.head()

,lat,lon,201805_NDVI,201805_SAVI,201805_EVI,201812_NDVI,201812_SAVI,201812_EVI,201801_NDVI,201801_SAVI,...,201810_EVI,201808_NDVI,201808_SAVI,201808_EVI,201809_NDVI,201809_SAVI,201809_EVI,201807_NDVI,201807_SAVI,201807_EVI
0,43.768661,-116.739350,0.176445,0.264642,0.561106,0.110320,0.165463,1.181932,-0.014664,-0.021996,...,2.157721,0.664816,0.997105,-7.735765,0.714488,1.071605,12.477374,0.653170,0.979619,-3.501948
1,43.768661,-116.739260,0.172219,0.258304,0.530505,0.100356,0.150518,1.080581,-0.018005,-0.027007,...,2.201001,0.665370,0.997933,-18.304278,0.692162,1.038121,11.431691,0.662800,0.994062,-2.991320
2,43.768661,-116.739170,0.166754,0.250107,0.514179,0.088017,0.132011,1.120817,-0.022291,-0.033435,...,2.114196,0.660143,0.990098,-19.423686,0.702319,1.053354,10.833210,0.647084,0.970498,-2.342274
3,43.768661,-116.739081,0.165201,0.247778,0.498756,0.108057,0.162068,1.569025,-0.022249,-0.033372,...,1.722504,0.658597,0.987778,-6.962047,0.704748,1.056998,12.353717,0.615602,0.923282,-4.086213
4,43.768661,-116.738991,0.165201,0.247778,0.498756,0.108057,0.162068,1.569025,-0.022249,-0.033372,...,1.722504,0.658597,0.987778,-6.962047,0.704748,1.056998,12.353717,0.615602,0.923282,-4.086213


In [40]:
satellite_imagery(source="Sentinel2_1C", center_lat=India_1_lat, center_lon=India_1_lon, edge_len=India_1_edge_len,
                  start_date='2018-1-01', end_date='2018-12-31',
                  plot_option='NDVI', time_series="monthly")

Total number of images in the collection:  94


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: UserWarning: Multiple tiles are selected. Proceed with caution.


Number of tiles selected:  2


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: There are partial images in the collection. Proceed with caution.


Number of partial images:  6


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:171: UserWarning: There are null values in the output DataFrame. Proceed with caution.
